## Code Details
Author: Rory Angus<br>
Created: 19JAN19<br>
Updated: 01APR19 - adjusted to add to the process as well as that the data has changed<br>
Version: 0.2<br>
***
This code loads the results data from Mongo and processes it to ensure that the correct elements are brought together. The end result is a single line that contains a journey, which has the results from the Diagnose and Measure survey.

It is important to note that my understanding of the data model has changed since the last code was written. The journey can have many steps and Clara survey results linked to it. In some instances, I have seen up to 8 different Clara results with a single journey. Checking with Shaofu, this is apparently normal, and so I am having to adjust this code to automate the gathering of a single set of data.
That is one user -> one journey -> one Clara result for the Diagnose step -> (optional) one Clara result for the Measure step

If there are more than two Clara results for step, then a decision needs to be made about which one to keep. I am not sure what that should be. I am guessing the first one for Diagnose and the latest one for Measure. This needs to be verified and certainly, it needs to be flagged in the reporting (new field).


# Package Importing + Variable Setting

In [1]:
import matplotlib
#need to use this otherwise nothing appears in the notebook from the charting point of view
matplotlib.use('module://ipykernel.pylab.backend_inline')
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from itertools import cycle, islice

import pandas as pd
import numpy as np
from math import pi
from math import ceil
from math import floor
import datetime

# mongo stuff
import pymongo
from pymongo import MongoClient
from bson.objectid import ObjectId

In [2]:
# if true the code outputs to the notebook a whole of diagnostic data that is helpful when writing but not so much when running it for real
verbose = False
# first run will truncate the target database and reload it from scratch. Once delta updates have been implmented this needs adjusting
first_run = True

# Set display options

In [3]:
# further details found by running:
# pd.describe_option('display')
# set the values to show all of the columns etc.
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', -1)  # or 199

# locals() # show all of the local environments

# Connect to Mongo DB

In [4]:
# create the connection to MongoDB
# define the location of the Mongo DB Server
# in this instance it is a local copy running on the dev machine. This is configurable at this point.
client = MongoClient('127.0.0.1', 27017)

# define what the database is called.
db = client.CLARA

# define the collection
#raw_data_collection = db.raw_data
raw_data_collection = db.raw_data_user_results

## Read Data 
This uses a variable that is defined above and puts it into a filter based on the student index. <br>
This needs to be replaced with the student ID.

In [5]:
# Query the data from the database using a filter
query_field = "userEmail"

# return results sorted
cursor = raw_data_collection.find().sort([(query_field, 1)])

# put the results into a dataframe
df = pd.DataFrame(list(cursor))

if verbose:
    print(df.shape)
    display(df.head())

In [6]:
if verbose:

    # count columns and rows
    print("Number of columns are " + str(len(df.columns)))
    print("Number of rows are " + str(len(df.index)))
    print()

    # output the shape of the dataframe
    print("The shape of the data frame is " + str(df.shape))
    print()

    # output the column names
    print("The column names of the data frame are: ")
    print(*df, sep='\n')
    print()

    # output the column names and datatypes
    print("The datatypes of the data frame are: ")
    print(df.dtypes)
    print()

## Remove the fields that we don't currently need

In [7]:
# get the unique users from data frame. This is what we will interate through
userId = df['userId'].unique()

if verbose:
    display(userId.shape)

# Calc Number of Surveys & Type per Journey

This goes through each user and looks at each journey then works out how many surveys in total for each journey. There can be 0, 1 or 2 to make it a valid data set. If there is more then there is an issue that will need to be manually investigated and corrected. 

It takes this further and works out how many diagnose and measure surveys per journey. This is to ensure that each journey has at only one of each type and that the logic takes into account that a journey with two surveys may not be correct if they are both diagnostic ones.

In [8]:
# set the counting variable up
userPos = 0

while userPos <= len(userId) - 1:
    # assign this users data to a temp dataframe for working with
    tudf = df[df['userId'] == userId[userPos]]

    # count number of rows of data returned for each user
    # this is a new feature that can be used to see how many surveys this users has done
    df.loc[df['userId'] == userId[userPos], 'numTotalClaraSurveys'] = len(tudf)

    # get the list of unique journey Id's for this user only
    journeyId = tudf['journeyId'].unique()

    # loop through the journeys for each user
    # remember that each journey can have one or more rows of data.
    journeyPos = 0
    while journeyPos <= len(journeyId) - 1:
        # assign this users data for a single journey to a temp dataframe for working with
        tjdf = df[df['journeyId'] == journeyId[journeyPos]]
        # count number of rows of data returned for each user per journey
        # this is a new feature that can be used to see how many surveys this users has done for each journey
        df.loc[((df['userId'] == userId[userPos]) &
                (df['journeyId'] == journeyId[journeyPos])
                ), 'numTotalClaraJourneySurveys'] = len(tjdf)
        # work out how many diagnose surveys per journey
        df.loc[((df['userId'] == userId[userPos]) &
                (df['journeyId'] == journeyId[journeyPos]) &
                (df['claraResultsJourneyStep'] == 'diagnose')
                ), 'numDiagnoseSurveysPerJourney'] = len(
                    tjdf[tjdf['claraResultsJourneyStep'] == 'diagnose'])
        # work out how many measure surveys per journey
        df.loc[((df['userId'] == userId[userPos]) &
                (df['journeyId'] == journeyId[journeyPos]) &
                (df['claraResultsJourneyStep'] == 'measure')
                ), 'numMeasureSurveysPerJourney'] = len(
                    tjdf[tjdf['claraResultsJourneyStep'] == 'measure'])

        # increment journey by 1
        journeyPos += 1

    # increment user by 1
    userPos += 1
#df

# d167cf61-44a0-442c-9105-2d73b7d1decc is a good test case

## Multiple Surveys Error Reporting
This section will display any rows that need to be manually managed

In [9]:
# look for data that have more than type of survey (diagnose or measure) per journey
dfError = df.loc[(
    (df['numTotalClaraJourneySurveys'] >= 3)
    &  # if there are three surveys, at least one is doubled up and needs investigating
    ((df['numDiagnoseSurveysPerJourney'] >= 2)
     |  ## use or because it may be either step that has more than the one
     (df['numMeasureSurveysPerJourney'] >= 2)))]

In [10]:
if len(dfError) > 0:
    print(
        "There is some data that you need to look at to work out which records to keep. The data in question is stored in the "
        "data frame called dfError and is presented here for ease.")
    display(dfError)

## Build new data frame containing linked surveys
This section takes the clean data and builds the data structure to hold the information

In [11]:
# finds all of the diagnose surveys that have a corresponding measure survey
# this also assumes that there is only one of each
# the ones with more need to be handled with exception code
dfDiagnose = df.loc[(
    (df['numTotalClaraJourneySurveys'] == 2) &
    (df['numDiagnoseSurveysPerJourney'] == 1)) | (
        (df['numTotalClaraJourneySurveys'] == 1
         ) &  # bring in all the surveys that only have a diagnose step
        (df['claraResultsJourneyStep'] == 'diagnose'))].copy()

dfMeasure = df.loc[((df['numTotalClaraJourneySurveys'] == 2) &
                    (df['numMeasureSurveysPerJourney'] == 1))].copy()

In [12]:
# drop the uneeded columns from the diagnose as they will be merged back into the dataframe

dfDiagnose.drop(['numMeasureSurveysPerJourney'],
                inplace=True,
                axis=1,
                errors='ignore')
# drop the uneeded columns from the measure as they will be merged back into the dataframe
dfMeasure.drop([
    'insertdate', 'journeyCreatedAt', 'journeyGoal', 'journeyPurpose',
    'journeyTitle', 'userClientUserId', 'userCoachId', 'userEmail',
    'numTotalClaraSurveys', 'numTotalClaraJourneySurveys',
    'numDiagnoseSurveysPerJourney', '_id', 'userExtraData',
    'userLanguagePreference', 'rowIndex', 'userAvatarSupplied',
    'userDeclaraLinked', 'userDeletedAt', 'userStatus', 'nameId',
    'primaryEmail'
],
               inplace=True,
               axis=1,
               errors='ignore')

In [13]:
# Rename column headings for the measure survey

colNames = [
    'measure_ClaraId', 'measure_ClaraResult1', 'measure_ClaraResult2',
    'measure_ClaraResult3', 'measure_ClaraResult4', 'measure_ClaraResult5',
    'measure_ClaraResult6', 'measure_ClaraResult7', 'measure_ClaraResult8',
    'measure_ClaraResultCompletedAt', 'measure_ClaraResultsCreatedAt',
    'measure_ClaraResultsJourneyStep', 'journeyId', 'userId',
    'numMeasureSurveysPerJourney'
]

dfMeasure.columns = colNames

In [14]:
if verbose:
    dfDiagnose.columns

In [15]:
# Rename column headings for the diagnose survey

colNames = [
    '_id', 'diagnose_ClaraId', 'diagnose_ClaraResult1',
    'diagnose_ClaraResult2', 'diagnose_ClaraResult3', 'diagnose_ClaraResult4',
    'diagnose_ClaraResult5', 'diagnose_ClaraResult6', 'diagnose_ClaraResult7',
    'diagnose_ClaraResult8', 'diagnose_ClaraResultCompletedAt',
    'diagnose_ClaraResultsCreatedAt', 'claraResultsJourneyStep', 'insertdate',
    'journeyCreatedAt', 'journeyGoal', 'journeyId', 'journeyPurpose',
    'journeyTitle', 'nameId', 'primaryEmail', 'rowIndex', 'userId',
    'numTotalClaraSurveys', 'numTotalClaraJourneySurveys',
    'numDiagnoseSurveysPerJourney'
]

dfDiagnose.columns = colNames

In [16]:
# merge the two datadrames diagnose and measure to end up with the data having the structure of 1 row per journey
# with the second survey completed if applicable
dfResult = pd.merge(
    dfDiagnose,
    dfMeasure,
    left_on=['journeyId', 'userId'],
    right_on=['journeyId', 'userId'],
    how='outer')

# reset index - just in case => makes it unique
dfResult = dfResult.reset_index(drop=True)

In [17]:
if verbose:
    dfResult.shape
    print(*dfResult, sep='\n')
    display(dfResult)

# Write to Mongo the file file

### To Do
The index need to be replaced with the unique identifier for the student

In [18]:
# Loop through the data frame and build a list
# the list will be used for a bulk update of MongoDB

# I am having to convert to strings for the intergers as Mongo cannot handle the int64 datatype.
# It also cant handle the conversion to int32 at the point of loading the rows, so string is the fall back position

# define the list to hold the data
clara_row = []

# loop through dataframe and create each item in the list
for index, row in dfResult.iterrows():
    clara_row.insert(
        index, {
            "rowIndex":
            index,
            "journeyId":
            dfResult['journeyId'].iloc[index],
            "journeyTitle":
            dfResult['journeyTitle'].iloc[index],
            "journeyPurpose":
            dfResult['journeyPurpose'].iloc[index],
            "journeyGoal":
            dfResult['journeyGoal'].iloc[index],
            "journeyCreatedAt":
            dfResult['journeyCreatedAt'].iloc[index],
            "diagnose_ClaraId":
            dfResult['diagnose_ClaraId'].iloc[index],
            "claraResultsJourneyStep":
            dfResult['claraResultsJourneyStep'].iloc[index],
            "diagnose_ClaraResultsCreatedAt":
            dfResult['diagnose_ClaraResultsCreatedAt'].iloc[index],
            "diagnose_ClaraResultCompletedAt":
            dfResult['diagnose_ClaraResultCompletedAt'].iloc[index],
            "diagnose_ClaraResult1":
            dfResult['diagnose_ClaraResult1'].iloc[index],
            "diagnose_ClaraResult2":
            dfResult['diagnose_ClaraResult2'].iloc[index],
            "diagnose_ClaraResult3":
            dfResult['diagnose_ClaraResult3'].iloc[index],
            "diagnose_ClaraResult4":
            dfResult['diagnose_ClaraResult4'].iloc[index],
            "diagnose_ClaraResult5":
            dfResult['diagnose_ClaraResult5'].iloc[index],
            "diagnose_ClaraResult6":
            dfResult['diagnose_ClaraResult6'].iloc[index],
            "diagnose_ClaraResult7":
            dfResult['diagnose_ClaraResult7'].iloc[index],
            "diagnose_ClaraResult8":
            dfResult['diagnose_ClaraResult8'].iloc[index],
            "measure_ClaraId":
            dfResult['measure_ClaraId'].iloc[index],
            "measure_ClaraResultsJourneyStep":
            dfResult['measure_ClaraResultsJourneyStep'].iloc[index],
            "measure_ClaraResultsCreatedAt":
            dfResult['measure_ClaraResultsCreatedAt'].iloc[index],
            "measure_ClaraResultCompletedAt":
            dfResult['measure_ClaraResultCompletedAt'].iloc[index],
            "measure_ClaraResult1":
            dfResult['measure_ClaraResult1'].iloc[index],
            "measure_ClaraResult2":
            dfResult['measure_ClaraResult2'].iloc[index],
            "measure_ClaraResult3":
            dfResult['measure_ClaraResult3'].iloc[index],
            "measure_ClaraResult4":
            dfResult['measure_ClaraResult4'].iloc[index],
            "measure_ClaraResult5":
            dfResult['measure_ClaraResult5'].iloc[index],
            "measure_ClaraResult6":
            dfResult['measure_ClaraResult6'].iloc[index],
            "measure_ClaraResult7":
            dfResult['measure_ClaraResult7'].iloc[index],
            "measure_ClaraResult8":
            dfResult['measure_ClaraResult8'].iloc[index],
            "numTotalClaraSurveys":
            dfResult['numTotalClaraSurveys'].iloc[index],
            "numTotalClaraJourneySurveys":
            dfResult['numTotalClaraJourneySurveys'].iloc[index],
            "userNameId":
            dfResult['nameId'].iloc[index],
            "userPrimaryEmail":
            dfResult['primaryEmail'].iloc[index],
            "userId":
            dfResult['userId'].iloc[index],
            "insertdate":
            datetime.datetime.utcnow()
        })

if verbose:
    print(clara_row[0])

# Connect to Mongo DB

In [19]:
# create the connection to MongoDB
# define the location of the Mongo DB Server
# in this instance it is a local copy running on the dev machine. This is configurable at this point.
client = MongoClient('127.0.0.1', 27017)

# define what the database is called.
db = client.CLARA

# define the collection
# This is the collection that the data will be saved to. This is the processed data and used later for the analysis
raw_data_collection = db.raw_data_combined_user_results

Command to clean the database if needed when running this code

In [20]:
# Delete the raw_data_collection - used for testing
if first_run:
    raw_data_collection.drop()

### Post the data to the database

In [21]:
# bulk update the database

raw_data_collection.insert_many(clara_row)

if verbose:
    print(raw_data_collection.inserted_ids)

## Create Index

In [22]:
# Create some indexes
result = []

result.append(
    raw_data_collection.create_index([('rowIndex', pymongo.ASCENDING)],
                                     unique=True))
result.append(
    raw_data_collection.create_index([('userId', pymongo.ASCENDING)],
                                     unique=False))
result.append(
    raw_data_collection.create_index([('userPrimaryEmail', pymongo.ASCENDING)],
                                     unique=False))
result.append(
    raw_data_collection.create_index([('userClientUserId', pymongo.ASCENDING)],
                                     unique=False))
result.append(
    raw_data_collection.create_index([('userNameId', pymongo.ASCENDING)],
                                     unique=False))
result.append(
    raw_data_collection.create_index([('journeyId', pymongo.ASCENDING)],
                                     unique=False))
result.append(
    raw_data_collection.create_index([('journeyCreatedAt', pymongo.ASCENDING)],
                                     unique=False))
result.append(
    raw_data_collection.create_index([('diagnose_ClaraId', pymongo.ASCENDING)],
                                     unique=False))
result.append(
    raw_data_collection.create_index(
        [('diagnose_ClaraResultsCreatedAt', pymongo.ASCENDING)], unique=False))
result.append(
    raw_data_collection.create_index(
        [('diagnose_ClaraResultCompletedAt', pymongo.ASCENDING)],
        unique=False))
result.append(
    raw_data_collection.create_index(
        [('diagnose_ClaraResultsStep', pymongo.ASCENDING)], unique=False))
result.append(
    raw_data_collection.create_index([('measure_ClaraId', pymongo.ASCENDING)],
                                     unique=False))
result.append(
    raw_data_collection.create_index(
        [('measure_ClaraResultsCreatedAt', pymongo.ASCENDING)], unique=False))
result.append(
    raw_data_collection.create_index(
        [('measure_ClaraResultCompletedAt', pymongo.ASCENDING)], unique=False))
result.append(
    raw_data_collection.create_index(
        [('measure_ClaraResultsStep', pymongo.ASCENDING)], unique=False))
result.append(
    raw_data_collection.create_index([('insertdate', pymongo.ASCENDING)],
                                     unique=False))

if verbose:
    print(result)